In [ ]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 11.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import time # -- 프레임 계산을 위해 사용


video_path = './streets.mp4' #-- 사용할 영상 경로
min_confidence = 0.5

def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, (800,600), fx=0.8, fy=0.8)
    height, width, channels = img.shape
    #cv2.imshow("Original Image", img)

    #-- 창 크기 설정
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    #-- 탐지한 객체의 클래스 예측 
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > min_confidence:
                # 탐지한 객체 박싱
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            print(i, label)
            color = colors[i] #-- 경계 상자 컬러 설정 / 단일 생상 사용시 (255,255,255)사용(B,G,R)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2.imshow("YOLO test", img)
    
#-- yolo 포맷 및 클래스명 불러오기
model_file = './yolov3-tiny.weights' #-- 본인 개발 환경에 맞게 변경할 것
config_file = './yolov3-tiny.cfg' #-- 본인 개발 환경에 맞게 변경할 것
net = cv2.dnn.readNet(model_file, config_file)

#-- GPU 사용
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

#-- 클래스(names파일) 오픈 / 본인 개발 환경에 맞게 변경할 것
classes = []
with open("./coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
# output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

#-- 비디오 활성화
# cap = cv2.VideoCapture(video_path) #-- 웹캠 사용시 video_path를 0 으로 변경
cap = cv2.VideoCapture(video_path)
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    #-- q 입력시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

0 truck: 70.17
1 truck: 52.60
2 bus: 87.38
4 car: 55.48
5 car: 73.72
=== A frame took 0.119 seconds
0 truck: 66.81
2 truck: 52.86
3 bus: 85.65
5 car: 63.84
6 car: 71.99
=== A frame took 0.113 seconds
0 truck: 78.31
2 bus: 56.12
3 truck: 60.52
4 bus: 86.10
6 car: 69.04
=== A frame took 0.050 seconds
1 truck: 57.72
3 bus: 53.26
4 bus: 84.57
6 car: 70.34
7 car: 52.64
8 car: 50.25
=== A frame took 0.049 seconds
0 truck: 66.15
1 bus: 51.57
2 bus: 84.03
4 car: 68.84
=== A frame took 0.051 seconds
0 truck: 60.04
1 bus: 50.58
2 bus: 80.69
4 car: 68.88
=== A frame took 0.046 seconds
0 truck: 65.78
1 bus: 67.31
3 person: 66.29
4 car: 66.87
5 car: 57.07
=== A frame took 0.042 seconds
0 truck: 63.39
2 bus: 64.97
3 person: 73.40
4 car: 63.62
5 car: 56.04
6 bird: 63.56
=== A frame took 0.039 seconds
0 truck: 67.56
1 person: 79.88
2 car: 54.86
3 car: 67.15
4 car: 57.62
5 bird: 63.69
=== A frame took 0.039 seconds
0 truck: 65.84
1 person: 72.25
2 car: 71.98
3 bird: 56.43
=== A frame took 0.042 seconds